In [1]:
import cdag
from collections import defaultdict
from itertools import chain

In [2]:
cdag.load_data()

KeyError: 'PADER '

In [ ]:
cdag.graph.search_nodes(naam='TKAPPEN')

In [ ]:
def group_descr(code):
    lst = [g for g in groups if g['code'] == code]
    if len(lst) == 0:
        print(f"onbekende groep {code}")
    elif len(lst) == 1:
        return lst[0]
    else:
        print(f"meerdere mogelijkheden voor groep {code}")

In [ ]:
def groups_for_user(username):
    return [gl['groepcode'] for gl in group_links if username in gl['linkcode']]

def ancestors_of_group(groupcode, visited=None, level=1):
    if visited is None:
        visited = defaultdict(int)
    result = []
    for edge in group_hier:
        parent, child = edge['parent'], edge['child']
        if visited[parent]: continue
        if child == groupcode:
            visited[parent] = 1
            result.append({'parent': parent, 'level': level})
            result.extend(parents(parent, visited, level+1))
    return result

def effective_groups_for_user(username):
    parents = groups_for_user(username)
    groups = list(chain.from_iterable(ancestors_of_group(g) for g in parents))
    for p in parents:
        groups.append({'parent': p, 'level': 0})
    interm = {}
    for g in groups:
        key = g['parent']
        if key in interm:
            interm[key] = min(interm[key], g['level'])
        else:
            interm[key] = g['level']
    result = [{'parent': key, 'level': value} for key, value in interm.items()]
    result.sort(key=lambda x: x['level'])
    return result

In [ ]:
group_types = defaultdict(int)
group_hier = []
for group_link in group_links:
    group_code, link_code = group_link['groepcode'], group_link['linkcode']
    first, second = link_code[0], link_code[1]
    if first.isalpha():
        group_types['a-z'] += 1
    elif first == '@':
        if group_link['islid'] != '1':
            print(group_link)
        group_hier.append({'parent': group_code, 'child': link_code[2:]})
        group_types[first+second] += 1
    else:
        group_types[first] += 1
# group_links: 14382 entries 'a-z', 3020 entries '0-9', 4630 entries '@G'

In [ ]:
def worksettings(code):
    return [w for w in worksetting if w['CODE'] == code]

In [ ]:
for p in effective_groups_for_user('TKAPPEN'):
    group_code = p['parent']
    ws = worksettings(group_code)
    if len(ws) > 0: # beperking tot diepe nesting: and p['level'] > 5:
        desc = group(group_code)
        print(f"groep {group_code} ({desc['searchcode']}), niveau={p['level']}")
        for e in ws:
            setting_id = e.get('SettingId', '')
            if setting_id == '__CSSTD__CSSTD__': continue # ignore this for the time being
            print(f"    {e.get('SegmentClassId', '')}\t{e.get('SegmentId', '')}\t{setting_id}")
        print('-----')